In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import linprog
from tabulate import tabulate

def tabprint(msg, A):
    print(msg)
    print(tabulate(A, tablefmt="fancy_grid"))

In [2]:
df = pd.read_csv('C:/Users/harini/Desktop/NEUCourses/ADM/Project/e_commerce_synthetic_data.csv')

In [3]:
df = df.drop('Unnamed: 0', axis = 1)

In [4]:
df.columns

Index(['user_id', 'product_id', 'ratings', 'user_session', 'campaign',
       'promotion', 'channel', 'discount', 'Street Address', 'City', 'State',
       'ZipCode', 'event_time', 'event_type', 'category_id', 'category_code',
       'brand', 'price', 'selling_price'],
      dtype='object')

In [5]:
df.event_type.value_counts()

view                5685348
cart                3272177
remove_from_cart    2237915
purchase             737314
Name: event_type, dtype: int64

In [6]:
df[df.event_type == 'purchase']

,user_id,product_id,ratings,user_session,campaign,promotion,channel,discount,Street Address,City,State,ZipCode,event_time,event_type,category_id,category_code,brand,price,selling_price
6,494077766,4958,1,c99a50e8-2fac-4c4d-89ec-41c05f114554,Social Media,Discount,Phone,10,57815 Cheryl Unions,Vanessaburgh,Oklahoma,97659,2019-12-01 00:26:15 UTC,purchase,1487580009471148064,NaN,runail,1.19,1.0710
16,494077766,5335,4,c99a50e8-2fac-4c4d-89ec-41c05f114554,Social Media,Discount,Phone,10,57815 Cheryl Unions,Vanessaburgh,Oklahoma,97659,2019-12-01 00:26:15 UTC,purchase,1487580009605365797,NaN,runail,0.40,0.3600
35,494077766,5761411,4,c99a50e8-2fac-4c4d-89ec-41c05f114554,Social Media,Discount,Phone,10,57815 Cheryl Unions,Vanessaburgh,Oklahoma,97659,2019-12-01 00:26:15 UTC,purchase,1487580007432717250,NaN,ingarden,2.86,2.5740
43,576802932,5742926,4,c2e3713b-9c29-4f87-a38c-03889c51dd38,Social Media,Buy One Get One,Desktop,0,604 Tucker Valley Suite 759,Sharonhaven,Ohio,67847,2019-12-01 01:30:47 UTC,purchase,1487580006317032337,NaN,yoko,21.78,21.7800
47,494077766,5666749,4,c99a50e8-2fac-4c4d-89ec-41c05f114554,Social Media,Discount,Phone,10,57815 Cheryl Unions,Vanessaburgh,Oklahoma,97659,2019-12-01 00:26:15 UTC,purchase,1487580009076883471,NaN,philips,3.43,3.0870
50,546170008,5862564,2,8d170406-e64d-412b-850e-3c4a32854837,E-mail,Free Sample,Desktop,0,65934 Lori Burgs,Toniville,Minnesota,52619,2019-10-26 07:11:52 UTC,purchase,1487580008145748965,NaN,roubloff,1.71,1.7100
74,576802932,5700046,5,c2e3713b-9c29-4f87-a38c-03889c51dd38,Social Media,Buy One Get One,Desktop,0,604 Tucker Valley Suite 759,Sharonhaven,Ohio,67847,2019-12-01 01:30:47 UTC,purchase,1487580009286598681,NaN,runail,0.40,0.4000
78,494077766,5810157,4,c99a50e8-2fac-4c4d-89ec-41c05f114554,Social Media,Discount,Phone,10,57815 Cheryl Unions,Vanessaburgh,Oklahoma,97659,2019-12-01 00:26:15 UTC,purchase,1487580009445982239,NaN,irisk,1.43,1.2870
136,579752970,5836528,1,62ae8d71-9453-4432-9256-8294c0dd71ee,E-mail,Discount,Phone,15,71427 Freeman Park Suite 075,Shawnland,Wyoming,13317,2020-01-19 18:46:58 UTC,purchase,1487580013522845895,NaN,nagaraku,0.95,0.8075
140,579752970,5836528,1,959ff64d-d9b5-4f12-b91c-3e8d3b2d9ec8,Social Media,Buy One Get One,Phone,0,71427 Freeman Park Suite 075,Shawnland,Wyoming,13317,2019-12-10 15:14:59 UTC,purchase,1487580013522845895,NaN,nagaraku,0.95,0.9500


In [7]:
a = df[df['user_session'] =='4cf6df21-216a-409e-ba43-e8c4d4bf0c6d'][['event_time','event_type','brand','product_id']]

In [8]:
df[df['user_session'] =='4cf6df21-216a-409e-ba43-e8c4d4bf0c6d'][['event_time']].max()

event_time    2019-11-04 14:37:04 UTC
dtype: object

In [9]:
df.columns

Index(['user_id', 'product_id', 'ratings', 'user_session', 'campaign',
       'promotion', 'channel', 'discount', 'Street Address', 'City', 'State',
       'ZipCode', 'event_time', 'event_type', 'category_id', 'category_code',
       'brand', 'price', 'selling_price'],
      dtype='object')

Pricing

In [10]:
purchased = df[df['event_type'] =='purchase'][['brand','product_id','event_time','promotion','discount','price','selling_price']]

In [11]:
purchased.head()

,brand,product_id,event_time,promotion,discount,price,selling_price
6,runail,4958,2019-12-01 00:26:15 UTC,Discount,10,1.19,1.071
16,runail,5335,2019-12-01 00:26:15 UTC,Discount,10,0.40,0.360
35,ingarden,5761411,2019-12-01 00:26:15 UTC,Discount,10,2.86,2.574
43,yoko,5742926,2019-12-01 01:30:47 UTC,Buy One Get One,0,21.78,21.780
47,philips,5666749,2019-12-01 00:26:15 UTC,Discount,10,3.43,3.087


In [12]:
purchased['month'] = pd.DatetimeIndex(purchased.event_time).month_name(locale = 'English')

In [13]:
purchased.head()

,brand,product_id,event_time,promotion,discount,price,selling_price,month
6,runail,4958,2019-12-01 00:26:15 UTC,Discount,10,1.19,1.071,December
16,runail,5335,2019-12-01 00:26:15 UTC,Discount,10,0.40,0.360,December
35,ingarden,5761411,2019-12-01 00:26:15 UTC,Discount,10,2.86,2.574,December
43,yoko,5742926,2019-12-01 01:30:47 UTC,Buy One Get One,0,21.78,21.780,December
47,philips,5666749,2019-12-01 00:26:15 UTC,Discount,10,3.43,3.087,December


In [14]:
grouped = purchased.groupby(['brand','product_id','month','discount'])

In [15]:
demand_df = grouped['selling_price'].count().reset_index()

In [16]:
price_df = grouped['selling_price'].sum().reset_index()

In [17]:
stock_df = demand_df.groupby(['brand','product_id',])['selling_price'].max().reset_index()
stock_df['selling_price'] = stock_df['selling_price'].apply(lambda x: x*2)

In [18]:
demand_df['month_disc'] = demand_df['month'].map(str) + '_' + demand_df['discount'].map(str)
price_df['month_disc'] = price_df['month'].map(str) + '_' + price_df['discount'].map(str)

In [19]:
column_names = ['month_disc']

mapper = {} 
for i, col_name in enumerate(column_names):
    for val in demand_df[col_name].unique():
        mapper[str(val) + str(i)] = len(mapper)
         
    
def one_hot(values):
    v = np.zeros( len(mapper) )
    for i, val in enumerate(values): 
        v[ mapper[str(val) + str(i)] ] = 1
    return v    
    
demand_df['one_hot'] = demand_df[column_names].values.tolist()
demand_df['one_hot'] = demand_df['one_hot'].map(one_hot)

In [20]:
column_names = ['month_disc']

mapper = {} 
for i, col_name in enumerate(column_names):
    for val in price_df[col_name].unique():
        mapper[str(val) + str(i)] = len(mapper)
         
    
def one_hot(values):
    v = np.zeros( len(mapper) )
    for i, val in enumerate(values): 
        v[ mapper[str(val) + str(i)] ] = 1
    return v    
    
price_df['one_hot'] = price_df[column_names].values.tolist()
price_df['one_hot'] = price_df['one_hot'].map(one_hot)

In [21]:
demand_df['one_hot'] = demand_df['one_hot']*demand_df['selling_price']
price_df['one_hot'] = price_df['one_hot']*price_df['selling_price']

In [22]:
demand_df.head()

,brand,product_id,month,discount,selling_price,month_disc,one_hot
0,airnails,5659639,December,0,3,December_0,"[3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,airnails,5659639,December,10,1,December_10,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,airnails,5659639,December,20,1,December_20,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,airnails,5659639,February,0,3,February_0,"[0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,airnails,5659639,February,10,1,February_10,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."


In [23]:
price_df.head()

,brand,product_id,month,discount,selling_price,month_disc,one_hot
0,airnails,5659639,December,0,2.850,December_0,"[2.8499999999999996, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,airnails,5659639,December,10,0.855,December_10,"[0.0, 0.855, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,airnails,5659639,December,20,0.760,December_20,"[0.0, 0.0, 0.76, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,airnails,5659639,February,0,2.850,February_0,"[0.0, 0.0, 0.0, 2.8499999999999996, 0.0, 0.0, ..."
4,airnails,5659639,February,10,0.855,February_10,"[0.0, 0.0, 0.0, 0.0, 0.855, 0.0, 0.0, 0.0, 0.0..."


In [24]:
def count_by_md(df):
    counters = np.zeros(len(demand_df['month_disc'].unique()))
    for one_hot in df['one_hot'].values:
        month_dis_id = np.argmax(one_hot)
        counters[month_dis_id]  =  one_hot[month_dis_id]
    return counters

In [26]:
result_pricing = []

for prod in demand_df.product_id.unique():
    
    pid = prod
    brand = demand_df[demand_df['product_id'] == pid]['brand'].unique()[0]
    pid_demand_df = demand_df[demand_df['product_id'] == pid].reset_index(drop = True)
    D = count_by_md(pid_demand_df)
    pid_price_df = price_df[price_df['product_id']== pid].reset_index(drop = True)
    P = count_by_md(pid_price_df)
    C = stock_df[stock_df['product_id']==pid]['selling_price'].reset_index(drop = True)
    
    
    L = len(demand_df['month_disc'].unique())

    M = np.array([[
        1 if i >= 4*j and i < 4*(j+1) else 0
        for i in range(L)
    ] for j in range(5)])
    
    B = [1 for i in range(5)]

    # Second, we ensure that the sum of all demands is less than the availbale stock level,
    # that is z.Df <= C
    Df = np.array(D).reshape(1, L)
    
    try:

        res = linprog(-np.array(P).flatten(), 
                      A_eq=M, 
                      b_eq=B, 
                      A_ub=Df, 
                      b_ub=np.array([C]), 
                      bounds=(0, None))
    
        if(res.success):
            result = np.array(res.x).tolist()
            result.insert(0,pid)
            result.insert(0,brand)
            result.append(-res.fun)
        result_pricing.append(result)
    except:
        continue

In [27]:
md = demand_df['month_disc'].unique().tolist()
md.insert(0,'Producr_id')
md.insert(0,'Brand')
md.append('net_revenue')

In [28]:
pricing_df=pd.DataFrame(result_pricing,columns=md)
len(pricing_df)

23079

In [29]:
pricing_df.to_csv('C:/Users/harini/Desktop/NEUCourses/ADM/Project/pricing_strategy_data.csv')